介绍有关梯度，GPU,内存的知识

如果 out=f(x)是标量，out.backward() 会计算 ∂out/∂x  
当out是矩阵时，out.backward() 默认不允许直接调用（因为梯度是矩阵，方向不明确）  
必须提供一个与 out 形状相同的张量 gradient（即 grad_outputs），表示 out 的每个元素对最终损失的贡献权重。  
torch.ones_like(out) 对 out 的每个元素求和（即 out.sum()）  
对求和后的标量结果调用 backward()

In [ ]:
import torch
import numpy as np
x = torch.tensor([[1., 2.], [3., 4.]], requires_grad=True)
y = torch.tensor([[5., 6.], [7., 8.]], requires_grad=True)#定义 x ，y并启用梯度跟踪
out = (x.pow(2)*y).sum()#对x的每个元素平方，再对所有元素求和
out.backward()#out最好是标量，不然很复杂
print(x.grad)#表示OUT对x的梯度
print(y.grad)

tensor([[10., 24.],
        [42., 64.]])
tensor([[ 1.,  4.],
        [ 9., 16.]])


数据放到GPU或者CPU上进行计算

In [ ]:
x = torch.tensor([1, 2])#默认是放在cpu  
if torch.cuda.is_available():         
    device = torch.device('cuda')     
    y = torch.ones_like(x, device=device)
    x = x.to(device)                  
    z = x + y                         
    print(z)                          
    print(z.to('cpu'), torch.double)  

tensor([2, 3], device='cuda:0')
tensor([2, 3]) torch.float64


张量连结（concatenate）提供张量列表，给出沿哪个轴连结

In [6]:
X = torch.arange(12, dtype=torch.float32).reshape((3,4))
Y = torch.tensor([[2.0, 1, 4, 3], [1, 2, 3, 4], [4, 3, 2, 1]])
torch.cat((X, Y), dim=0), torch.cat((X, Y), dim=1)

(tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [ 2.,  1.,  4.,  3.],
         [ 1.,  2.,  3.,  4.],
         [ 4.,  3.,  2.,  1.]]),
 tensor([[ 0.,  1.,  2.,  3.,  2.,  1.,  4.,  3.],
         [ 4.,  5.,  6.,  7.,  1.,  2.,  3.,  4.],
         [ 8.,  9., 10., 11.,  4.,  3.,  2.,  1.]]))

节省内存  
Python首先计算Y + X，为结果分配新的内存，然后使Y指向内存中的这个新位置

In [7]:
before = id(Y)
Y = Y + X
id(Y) == before

False

执行原地更新，即Y的地址保持不变，只是更新上面的值  
1.切片表示法Y[:] = <expression>
2.+=操作

In [8]:
Z = torch.zeros_like(Y)
print('id(Z):', id(Z))
Z[:] = X + Y
print('id(Z):', id(Z))


id(Z): 128033535797152
id(Z): 128033535797152


In [9]:
before = id(Y)
Y += X
id(Y) == before

True